<a href="https://colab.research.google.com/github/Spandana365/3_Bias-Busters/blob/main/biased_distil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#gender_bias_hiring (1).csv

In [24]:
# Install necessary packages
!pip install transformers torch scikit-learn pandas numpy

In [27]:

import torch
import pandas as pd
import numpy as np
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score
from fairlearn.reductions import ExponentiatedGradient, DemographicParity, EqualizedOdds
from fairlearn.metrics import demographic_parity_difference, equalized_odds_difference

In [28]:
from sklearn.base import BaseEstimater, ClassifierMixin

ImportError: cannot import name 'BaseEstimater' from 'sklearn.base' (/usr/local/lib/python3.11/dist-packages/sklearn/base.py)

In [29]:

# Upload the dataset to Google Colab
from google.colab import files
uploaded = files.upload()

# Load the dataset
df = pd.read_csv('gender_bias_hiring (1).csv')

Saving gender_bias_hiring (1).csv to gender_bias_hiring (1) (1).csv


In [30]:


# Drop unwanted columns
df = df.drop(columns=['AI Score', 'Name'], errors='ignore')  # Ignore if not present

# Combine categorical text columns into a single text input
df['Text'] = df[['Job Role', 'Skills', 'Education', 'Certification', 'Gender']].astype(str).agg(' '.join, axis=1)

# Define numerical features & target
num_features = ['Experience', 'Salary Expectation', 'Project Count']
X_num = df[num_features].fillna(0)  # Fill missing values
y = df['Recruiter Decision'].map({'Not Hired': 0, 'Hired': 1})  # Convert labels to 0/1

# Standardize numerical features
scaler = StandardScaler()
X_num_scaled = scaler.fit_transform(X_num)

In [31]:

# Load DistilBERT tokenizer & model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
bert_model = DistilBertModel.from_pretrained('distilbert-base-uncased')

# Tokenize and encode text data
def get_bert_embeddings(text_list):
    inputs = tokenizer(text_list, padding=True, truncation=True, return_tensors="pt", max_length=50)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state[:, 0, :].numpy()  # Take [CLS] token embeddings

In [32]:
X_text = get_bert_embeddings(df['Text'].tolist())

# Combine numerical features and text embeddings
X_combined = np.hstack((X_num_scaled, X_text))

# Sensitive feature (Gender)
sensitive_feature = df['Gender'].map({'Male': 1, 'Female': 0})  # Convert to binary

# Split dataset
X_train, X_test, y_train, y_test, S_train, S_test = train_test_split(X_combined, y, sensitive_feature, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Create DataLoader
batch_size = 16
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Define Neural Network Classifier
class Classifier(nn.Module):
    def __init__(self, input_dim):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 2)  # Output layer (binary classification)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)  # No activation (CrossEntropyLoss applies softmax)
        return x


In [ ]:

# **Step 1: Wrap PyTorch Model in an Scikit-Learn Estimator**
class SklearnWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, model, epochs=10, batch_size=16, learning_rate=0.001):
        self.model = model
        self.epochs = epochs
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)

    def fit(self, X, y):
        X_tensor = torch.tensor(X, dtype=torch.float32)
        y_tensor = torch.tensor(y.values, dtype=torch.long)

        dataset = TensorDataset(X_tensor, y_tensor)
        train_loader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True)

        self.model.train()
        for epoch in range(self.epochs):
            total_loss = 0
            for batch_X, batch_y in train_loader:
                self.optimizer.zero_grad()
                outputs = self.model(batch_X)
                loss = self.criterion(outputs, batch_y)
                loss.backward()
                self.optimizer.step()
                total_loss += loss.item()
            print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")

    def predict(self, X):
        X_tensor = torch.tensor(X, dtype=torch.float32)
        self.model.eval()
        with torch.no_grad():
            outputs = self.model(X_tensor)
            predictions = torch.argmax(outputs, dim=1)
        return predictions.numpy()

    def predict_proba(self, X):
        X_tensor = torch.tensor(X, dtype=torch.float32)
        self.model.eval()
        with torch.no_grad():
            outputs = self.model(X_tensor)
            probs = torch.nn.functional.softmax(outputs, dim=1)
        return probs.numpy()

In [76]:

# Initialize model
input_dim = X_combined.shape[1]

pytorch_model = Classifier(input_dim)

# **Step 4: Evaluate Initial Model**
predictions = sklearn_model.predict(X_test)
initial_accuracy = accuracy_score(y_test, predictions)
print(f"Initial Model Accuracy: {initial_accuracy:.4f}")

# Train model
epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")

# Evaluate model
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    predictions = torch.argmax(test_outputs, dim=1)
    accuracy = (predictions == y_test_tensor).float().mean().item()

print(f"Model Accuracy: {accuracy:.4f}")


Initial Model Accuracy: 0.4900
Epoch 1, Loss: 0.6660011398792267
Epoch 2, Loss: 0.6686079168319702
Epoch 3, Loss: 0.6664910173416138
Epoch 4, Loss: 0.6717152547836304
Epoch 5, Loss: 0.6669829702377319
Epoch 6, Loss: 0.6624868035316467
Epoch 7, Loss: 0.6731626105308532
Epoch 8, Loss: 0.6645985651016235
Epoch 9, Loss: 0.6631394731998443
Epoch 10, Loss: 0.6720297861099244
Model Accuracy: 0.5600


In [39]:
# Function to preprocess and predict a new candidate
def predict_candidate(model, job_role, skills, education, certification, gender, experience, salary_expectation, project_count):
    # Combine text features
    text_input = f"{job_role} {skills} {education} {certification} {gender}"

    # Convert text into DistilBERT embeddings
    text_embedding = get_bert_embeddings([text_input])  # Shape: (1, 768)

    # Convert numerical features into a NumPy array
    num_input = np.array([[experience, salary_expectation, project_count]])  # Shape: (1, 3)

    num_input_scaled = scaler.transform(pd.DataFrame(num_input, columns=num_features))  # Keep feature names


    # Combine numerical and text features
    input_combined = np.hstack((num_input_scaled, text_embedding))  # Shape: (1, 771)

    # Convert to PyTorch tensor
    input_tensor = torch.tensor(input_combined, dtype=torch.float32)

    # Make prediction
    model.eval()
    with torch.no_grad():
        output = model(input_tensor)
        prediction = torch.argmax(output, dim=1).item()

    return "Hired" if prediction == 1 else "Not Hired"


# Define same candidate details for both male & female
job_role = "Software Engineer"
skills = "Python, Machine Learning, TensorFlow"
education = "Master's"
certification = "Google Cloud Certified"
experience = 5  # Years of experience
salary_expectation = 60000  # Expected salary
project_count = 4  # Number of projects

# Predict for Male Candidate
male_prediction = predict_candidate(model, job_role, skills, education, certification, "Male", experience, salary_expectation, project_count)
print("Prediction for Male Candidate:", male_prediction)

# Predict for Female Candidate
female_prediction = predict_candidate(model, job_role, skills, education, certification, "Female", experience, salary_expectation, project_count)
print("Prediction for Female Candidate:", female_prediction)

# Check for bias
if male_prediction == "Hired" and female_prediction == "Not Hired":
    print("⚠️ Gender Bias Detected: The model hires the male but not the female!")
elif male_prediction == female_prediction:
    print("✅ No Gender Bias: The model makes the same decision for both candidates.")
else:
    print("🔍 Partial Bias: The model makes different decisions, but not necessarily biased.")


Prediction for Male Candidate: Hired
Prediction for Female Candidate: Not Hired
⚠️ Gender Bias Detected: The model hires the male but not the female!


In [40]:
!pip install fairlearn

In [14]:


import torch
import numpy as np
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertModel
from fairlearn.reductions import ExponentiatedGradient, DemographicParity
from fairlearn.metrics import demographic_parity_difference, equalized_odds_difference
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset

In [77]:
# Evaluate model
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    predictions = torch.argmax(test_outputs, dim=1)
    accuracy = accuracy_score(y_test, predictions)

print(f"Model Accuracy (Before Bias Mitigation): {accuracy:.4f}")

# Measure Bias Before Mitigation
dpd_before = demographic_parity_difference(y_test, predictions, sensitive_features=S_test)
eod_before = equalized_odds_difference(y_test, predictions, sensitive_features=S_test)

print(f"⚠ Bias Before Mitigation - Demographic Parity Difference: {dpd_before:.4f}")
print(f"⚠ Bias Before Mitigation - Equalized Odds Difference: {eod_before:.4f}")


Model Accuracy (Before Bias Mitigation): 0.5600
⚠ Bias Before Mitigation - Demographic Parity Difference: 0.8446
⚠ Bias Before Mitigation - Equalized Odds Difference: 0.8710


In [42]:

from fairlearn.metrics import demographic_parity_difference, equalized_odds_difference
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.base import BaseEstimator, ClassifierMixin


In [67]:

# **Step 1: Wrap PyTorch Model in an Scikit-Learn Estimator**
class SklearnWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, model, epochs=10, batch_size=16, learning_rate=0.001):
        self.model = model
        self.epochs = epochs
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)

    def fit(self, X, y):
        X_tensor = torch.tensor(X, dtype=torch.float32)
        y_tensor = torch.tensor(y.values, dtype=torch.long)

        dataset = TensorDataset(X_tensor, y_tensor)
        train_loader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True)

        self.model.train()
        for epoch in range(self.epochs):
            total_loss = 0
            for batch_X, batch_y in train_loader:
                self.optimizer.zero_grad()
                outputs = self.model(batch_X)
                loss = self.criterion(outputs, batch_y)
                loss.backward()
                self.optimizer.step()
                total_loss += loss.item()
            print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")

    def predict(self, X):
        X_tensor = torch.tensor(X, dtype=torch.float32)
        self.model.eval()
        with torch.no_grad():
            outputs = self.model(X_tensor)
            predictions = torch.argmax(outputs, dim=1)
        return predictions.cpu().numpy()

    def predict_proba(self, X):
        X_tensor = torch.tensor(X, dtype=torch.float32)
        self.model.eval()
        with torch.no_grad():
            outputs = self.model(X_tensor)
            probs = torch.nn.functional.softmax(outputs, dim=1)
        return probs.numpy()

# **Step 2: Initialize Wrapped Model**
input_dim = X_combined.shape[1]
pytorch_model = Classifier(input_dim)
sklearn_model = SklearnWrapper(pytorch_model, epochs=10)

# **Step 3: Train the Model**
sklearn_model.fit(X_train, y_train)

# **Step 4: Evaluate Initial Model**
predictions = sklearn_model.predict(X_test)
initial_accuracy = accuracy_score(y_test, predictions)
print(f"Initial Model Accuracy: {initial_accuracy:.4f}")


Epoch 1, Loss: 0.7001223635673522
Epoch 2, Loss: 0.6936214184761047
Epoch 3, Loss: 0.6939630627632141
Epoch 4, Loss: 0.6928565919399261
Epoch 5, Loss: 0.6890280294418335
Epoch 6, Loss: 0.6893407118320465
Epoch 7, Loss: 0.6893762457370758
Epoch 8, Loss: 0.6822642350196838
Epoch 9, Loss: 0.6843883407115936
Epoch 10, Loss: 0.681630345582962
Initial Model Accuracy: 0.4900


In [56]:
from sklearn.linear_model import LogisticRegression

# Extract DistilBERT embeddings (X_train) and PyTorch model’s predictions
X_train_np = sklearn_model.predict_proba(X_train)  # Get probability outputs
X_test_np = sklearn_model.predict_proba(X_test)  # Get test probabilities

# Train a Logistic Regression model on PyTorch's predictions
log_reg = LogisticRegression(max_iter=2000)
log_reg.fit(X_train_np, y_train)  # Train Logistic Regression on embeddings


LogisticRegression(max_iter=2000)

In [93]:
fair_model = ExponentiatedGradient(estimator=log_reg, constraints=DemographicParity())


In [94]:

try:
    fair_model.fit(X_train, y_train, sensitive_features=S_train)
except Exception as e:
    print("⚠ Training Error:", str(e))

# Predict with Fair Model
fair_predictions = fair_model.predict(X_test).astype(int)  # Ensure integer class labels


# Evaluate Fair Model Accuracy
fair_accuracy = accuracy_score(y_test, fair_predictions)
print(f"✅ Fair Model Accuracy: {fair_accuracy:.4f}")

# Measure Bias After Mitigation
dpd_fair = demographic_parity_difference(y_test, fair_predictions, sensitive_features=S_test)
eod_fair = equalized_odds_difference(y_test, fair_predictions, sensitive_features=S_test)


print("\n✅ Fair Model - Bias Scores:")
print(f"✔️ Demographic Parity Difference: {dpd_fair:.4f}")
print(f"✔️ Equalized Odds Difference: {eod_fair:.4f}")

✅ Fair Model Accuracy: 0.4700

✅ Fair Model - Bias Scores:
✔️ Demographic Parity Difference: 0.0296
✔️ Equalized Odds Difference: 0.0298
